In [ ]:
import pandas as pd
import numpy as np
import random

In [117]:
import datetime
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv("train_foreclosure.csv")

In [4]:
lms = pd.read_excel("LMS_31JAN2019.xlsx")

In [5]:
val_msk = np.random.rand(len(data)) < 0.8

In [28]:
train = data[val_msk]
val = data[~val_msk]

In [ ]:
# lms["disbursal_percent"] = lms["NET_DISBURSED_AMT"] / lms["LOAN_AMT"]

In [7]:
# lms["delta_roi"] = lms["ORIGNAL_ROI"] -  lms["CURRENT_ROI"]

In [ ]:
# lms["last_emi_days"] = pd.to_datetime("2019-01-31") - check["LAST_RECEIPT_DATE"]

In [ ]:
# lms["interest_days"] = pd.to_datetime("2019-01-31") - check["INTEREST_START_DATE"]

In [ ]:
# lms["authorization_days"] = pd.to_datetime("2019-01-31") - check["AUTHORIZATIONDATE"]

In [ ]:
# lms = lms.drop(['LAST_RECEIPT_DATE', 'INTEREST_START_DATE', "AUTHORIZATIONDATE", "NET_DISBURSED_AMT", "ORIGNAL_ROI"], axis=1)

In [238]:
def preprocess(df, lms):
    lms["rem_percent"] = lms["PAID_PRINCIPAL"] / lms["LOAN_AMT"]
    lms["delta_roi"] = lms["ORIGNAL_ROI"] -  lms["CURRENT_ROI"]
    lms["last_emi_days"] = pd.to_datetime("2019-01-31") - lms["LAST_RECEIPT_DATE"]
    lms["interest_days"] = pd.to_datetime("2019-01-31") - lms["INTEREST_START_DATE"]
    lms["authorization_days"] = pd.to_datetime("2019-01-31") - lms["AUTHORIZATIONDATE"]
    lms = lms.drop(["PRODUCT","NPA_IN_LAST_MONTH","NPA_IN_CURRENT_MONTH",'CITY','LAST_RECEIPT_DATE', 'INTEREST_START_DATE', "AUTHORIZATIONDATE"], axis=1)
    new_df = pd.merge(df, lms, on="AGREEMENTID", how="left")
    new_df["last_emi_days"] = new_df["last_emi_days"].dt.days
    new_df["interest_days"] = new_df["interest_days"].dt.days
    new_df["authorization_days"] = new_df["authorization_days"].dt.days
    new_df = new_df.drop([ "CUSTOMERID"], axis =1)
    return new_df
    

In [9]:
lms_cp = lms.copy()

In [239]:
new_data = preprocess(data, lms_cp)

In [289]:
 X_train, X_test, y_train, y_test = train_test_split(new_data.drop(["FORECLOSURE"], axis=1), new_data["FORECLOSURE"], test_size=0.33, random_state=34)

In [106]:
# train = preprocess(train, lms_cp)

In [107]:
# val = preprocess(val, lms_cp)

In [108]:
# y_train = train["FORECLOSURE"]
# y_test =  val["FORECLOSURE"]
# X_train = train.drop(["FORECLOSURE", "CUSTOMERID"], axis=1)
# X_test = val.drop(["FORECLOSURE", "CUSTOMERID"], axis=1)
# X_train = X_train.set_index(["AGREEMENTID"])
# X_test = X_test.set_index(["AGREEMENTID"])

### XGBOOAST

In [318]:
import xgboost

In [319]:
# fit model no training data
model = xgboost.XGBClassifier(scale_pos_weight=10)
model.fit(X_train.drop(["AGREEMENTID"],axis=1), y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=10, seed=None,
       silent=True, subsample=1)

In [320]:
feat_imp = list(model.feature_importances_)
importance = {}
for idx, column in enumerate(X_train.drop(["AGREEMENTID"],axis=1).columns):
    importance[column] =  feat_imp[idx]
sorted(importance.items(), key=lambda kv: -kv[1])

[('SCHEMEID', 0.38964587),
 ('interest_days', 0.30798167),
 ('last_emi_days', 0.23261076),
 ('CURRENT_ROI', 0.028399546),
 ('ORIGNAL_TENOR', 0.012996014),
 ('NET_RECEIVABLE', 0.0072159735),
 ('delta_roi', 0.0055733025),
 ('EMI_RECEIVED_AMT', 0.0031244764),
 ('FOIR', 0.001768977),
 ('LOAN_AMT', 0.0015911547),
 ('LAST_RECEIPT_AMOUNT', 0.0014610533),
 ('PRE_EMI_DUEAMT', 0.0014394774),
 ('rem_percent', 0.0013697915),
 ('authorization_days', 0.001074465),
 ('CURRENT_TENOR', 0.000989538),
 ('ORIGNAL_ROI', 0.0007060814),
 ('PRE_EMI_RECEIVED_AMT', 0.0006953048),
 ('PAID_INTEREST', 0.00068345515),
 ('NET_LTV', 0.00067305483),
 ('NET_DISBURSED_AMT', 0.0),
 ('DUEDAY', 0.0),
 ('PRE_EMI_OS_AMOUNT', 0.0),
 ('EMI_DUEAMT', 0.0),
 ('EMI_OS_AMOUNT', 0.0),
 ('EXCESS_AVAILABLE', 0.0),
 ('EXCESS_ADJUSTED_AMT', 0.0),
 ('BALANCE_EXCESS', 0.0),
 ('OUTSTANDING_PRINCIPAL', 0.0),
 ('PAID_PRINCIPAL', 0.0),
 ('MONTHOPENING', 0.0),
 ('COMPLETED_TENURE', 0.0),
 ('BALANCE_TENURE', 0.0),
 ('DPD', 0.0),
 ('MOB', 0.0),


In [321]:
### make predictions for test data
y_pred = model.predict(X_test.drop(["AGREEMENTID"],axis=1))
predictions1 = y_pred

In [322]:
accuracy = metrics.accuracy_score(y_test, predictions1)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 65.49%


In [323]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, predictions1)
auc = metrics.auc(fpr, tpr)
print ("AUC :", auc)

AUC : 0.7559798996172769


### LIGHTGBM

In [324]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

In [325]:

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train.drop(["AGREEMENTID"],axis=1), y_train)
lgb_eval = lgb.Dataset(X_test.drop(["AGREEMENTID"],axis=1), y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=200,
                valid_sets=lgb_eval,
                early_stopping_rounds=15)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')




Starting training...
[1]	valid_0's l2: 0.0942457	valid_0's l1: 0.195239
Training until validation scores don't improve for 15 rounds.
[2]	valid_0's l2: 0.0887895	valid_0's l1: 0.188926
[3]	valid_0's l2: 0.0844091	valid_0's l1: 0.183437
[4]	valid_0's l2: 0.0803484	valid_0's l1: 0.17785
[5]	valid_0's l2: 0.0771505	valid_0's l1: 0.173054
[6]	valid_0's l2: 0.0743595	valid_0's l1: 0.168512
[7]	valid_0's l2: 0.071964	valid_0's l1: 0.164331
[8]	valid_0's l2: 0.0700332	valid_0's l1: 0.16059
[9]	valid_0's l2: 0.0684096	valid_0's l1: 0.157204
[10]	valid_0's l2: 0.0669312	valid_0's l1: 0.153954
[11]	valid_0's l2: 0.0657318	valid_0's l1: 0.151124
[12]	valid_0's l2: 0.0646744	valid_0's l1: 0.148576
[13]	valid_0's l2: 0.0636655	valid_0's l1: 0.145966
[14]	valid_0's l2: 0.0626133	valid_0's l1: 0.143458
[15]	valid_0's l2: 0.0618876	valid_0's l1: 0.14147
[16]	valid_0's l2: 0.0611261	valid_0's l1: 0.139497
[17]	valid_0's l2: 0.0605711	valid_0's l1: 0.137882
[18]	valid_0's l2: 0.0599727	valid_0's l1: 0.1

[157]	valid_0's l2: 0.0346689	valid_0's l1: 0.0925973
[158]	valid_0's l2: 0.0346078	valid_0's l1: 0.092496
[159]	valid_0's l2: 0.034498	valid_0's l1: 0.0923625
[160]	valid_0's l2: 0.0344255	valid_0's l1: 0.0922802
[161]	valid_0's l2: 0.0343787	valid_0's l1: 0.0922188
[162]	valid_0's l2: 0.034339	valid_0's l1: 0.092136
[163]	valid_0's l2: 0.0342892	valid_0's l1: 0.0920597
[164]	valid_0's l2: 0.0342087	valid_0's l1: 0.091967
[165]	valid_0's l2: 0.0340076	valid_0's l1: 0.0916657
[166]	valid_0's l2: 0.0338914	valid_0's l1: 0.0915089
[167]	valid_0's l2: 0.0337688	valid_0's l1: 0.0913376
[168]	valid_0's l2: 0.0336028	valid_0's l1: 0.0911091
[169]	valid_0's l2: 0.033528	valid_0's l1: 0.0909855
[170]	valid_0's l2: 0.033464	valid_0's l1: 0.0908935
[171]	valid_0's l2: 0.0333601	valid_0's l1: 0.0907536
[172]	valid_0's l2: 0.0332542	valid_0's l1: 0.090582
[173]	valid_0's l2: 0.0331402	valid_0's l1: 0.090441
[174]	valid_0's l2: 0.0330651	valid_0's l1: 0.0903472
[175]	valid_0's l2: 0.0329853	valid_0

In [326]:
print('Starting predicting...')
# predict
predictions2 = gbm.predict(X_test.drop(["AGREEMENTID"],axis=1),num_iteration=gbm.best_iteration )


Starting predicting...


In [327]:
# eval
fpr, tpr, thresholds = metrics.roc_curve(y_test, predictions2)
auc = metrics.auc(fpr, tpr)
print('The auc of prediction is:', auc)

The auc of prediction is: 0.9858454382279377


### Stacking

In [330]:
a = 0.1
b = 1- b
final_pred =  np.maximum(predictions1, predictions2) #(a*predictions1) + (b*predictions2)
fpr, tpr, thresholds = metrics.roc_curve(y_test, final_pred)
auc = metrics.auc(fpr, tpr)
print ("AUC :", auc)

AUC : 0.7970096259956487


In [288]:
predictions1

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [338]:
test = pd.read_csv("test_foreclosure.csv")

In [339]:
test = preprocess(test, lms_cp)

In [340]:
test.columns

Index(['AGREEMENTID', 'FORECLOSURE', 'LOAN_AMT', 'NET_DISBURSED_AMT',
       'CURRENT_ROI', 'ORIGNAL_ROI', 'CURRENT_TENOR', 'ORIGNAL_TENOR',
       'DUEDAY', 'PRE_EMI_DUEAMT', 'PRE_EMI_RECEIVED_AMT', 'PRE_EMI_OS_AMOUNT',
       'EMI_DUEAMT', 'EMI_RECEIVED_AMT', 'EMI_OS_AMOUNT', 'EXCESS_AVAILABLE',
       'EXCESS_ADJUSTED_AMT', 'BALANCE_EXCESS', 'NET_RECEIVABLE',
       'OUTSTANDING_PRINCIPAL', 'PAID_PRINCIPAL', 'PAID_INTEREST',
       'MONTHOPENING', 'LAST_RECEIPT_AMOUNT', 'NET_LTV', 'COMPLETED_TENURE',
       'BALANCE_TENURE', 'DPD', 'FOIR', 'SCHEMEID', 'MOB', 'disbursal_percent',
       'delta_roi', 'last_emi_days', 'interest_days', 'authorization_days',
       'rem_percent'],
      dtype='object')

In [341]:
# make predictions for test data
predictions1 = model.predict(test.drop(["AGREEMENTID","FORECLOSURE"], axis=1))

In [342]:
# make predictions for test data
predictions2 = gbm.predict(test.drop(["AGREEMENTID","FORECLOSURE"], axis=1), num_iteration=gbm.best_iteration)

In [343]:
final_pred = (a*predictions1) + (b*predictions2)
test["FORECLOSURE"] = predictions2

In [344]:
test = test[["AGREEMENTID", "FORECLOSURE"]]
test.to_csv("output.csv", header=True, index=False)